In [ ]:
# Reference (resource from Youtube)
# Linkedin Message Automation with Selenium two parts https://www.youtube.com/watch?v=XdFUpFUDt88
# LinkedIn Contacts Automation with Selenium - Step by Step Bot https://www.youtube.com/watch?v=7aIb6iQZkDw&t=125s

In [ ]:
!pip install selenium
!pip install beautifulsoup4
# download chromdriver and put it in driver folder
# https://chromedriver.chromium.org/downloads

In [ ]:
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup

In [ ]:
browser = webdriver.Chrome('driver/chromedriver')

In [ ]:
login_page = 'https://www.linkedin.com/login' 

In [ ]:
browser.get(login_page)
time.sleep(2)

In [ ]:
# config.txt should include username and password in two lines
file = open('config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

In [ ]:
# target username input field
element_id = browser.find_element('id', 'username')
element_id.send_keys(username)
# target password input field
element_id = browser.find_element('id','password')
element_id.send_keys(password)
# press submit
element_id.submit()

In [ ]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
import pandas as pd
# there should be a csv file named "list.csv" containing contacts in the same directory
# read csv file
df = pd.read_csv('list.csv')
print(df)
# make sure indexes pair with number of rows
df = df.reset_index() 
res = []
for index, row in df.iterrows():
    print(index, '-------------------------------------------')
    # extract first name and url
    name = row['Name'].split()[0]
    url = row['LinkedIn']
    # optional to add a note
    # message = "Hi " + name + "! I would love to connect!"
    # print(message)
    try:
        # go to their linkedin page
        browser.get(url) 
        print(name)
        time.sleep(2)
    except Exception as e:
        # to do: catch url not found error
        print("error: page not found " , row['Name'], row['LinkedIn'], e)
        res.append(row)
        continue
    try:
        # target the card section containing the set of buttons 
        top_card = browser.find_element(By.CLASS_NAME, "pv-top-card-v2-ctas")
        # location the set of buttons
        all_buttons = top_card.find_elements(By.TAG_NAME, "button");
        num_btn = len(all_buttons)
        print("all_buttons", len(all_buttons))
        connect_btn = [btn for btn in all_buttons if btn.text == "Connect"]
        connect_btn[0].click()
        print("- Connect")
        # some user profiles have "Connect" button and others have "Message"/"Follow" button
        # for the second case, click Connect through clicking "More" first
    except Exception as e:
        # find "More" button and then select "Connect" in dropdown menu
        print("can't find Connect, go to More: ", e)
        #target "More" button
        more_btn = [btn for btn in all_buttons if btn.text == "More"]
        more_btn[0].click()
        print("found More")
        # target "Connect" button in dropdown
        dropdown_area = browser.find_element(By.CLASS_NAME, 'pvs-overflow-actions-dropdown__content')
        all_dropdowns = dropdown_area.find_elements(By.CLASS_NAME, "artdeco-dropdown__item"); 
        length = len(all_dropdowns) 
        print("length",length)
        time.sleep(2)
        # tricky part
        path = "/html/body/div[" + str(4) + "]/div[3]/div/div/div/div[2]/div/div/main/section[1]/div[2]/div[3]/div/div[" + str(2+num_btn-1) + "]/div/div/ul/li[" + str(length) + "]/div"
        print(path)
        WebDriverWait(browser, 30).until(EC.element_to_be_clickable((By.XPATH, path))).click()
    finally:
        # for users that have "Connect" button right on their profile, the next step is to click "Send" directly
        try:
            # target Send button
            browser.find_element(By.XPATH, "//button[@aria-label='Send now']").click()
            # remove this row from list
            # df.drop(df.index[index])
        except Exception as e: 
            print("couldn't find Send: ", e)
            # linkedin might prompt "How did you meet them?", custom your selection here
            browser.find_element(By.XPATH, "//button[@aria-label='Met at a work-related event']").click()
            time.sleep(2)
            # target Connect
            browser.find_element(By.XPATH, "//button[@aria-label='Connect']").click()
            time.sleep(2)
            # target Send
            browser.find_element(By.XPATH, "//button[@aria-label='Send now']").click()
            # now res should include all the users that were not added successfully
       